In [1]:
import random
import numpy as np
from collections import defaultdict

# Simple Blackjack environment
class BlackjackEnv:
    def __init__(self):
        self.deck = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 10, 10] * 4
        self.player = []
        self.dealer = []
    
    def draw_card(self):
        """Draw a random card from the deck."""
        return random.choice(self.deck)
    
    def hand_value(self, hand):
        """Calculate the value of a hand."""
        value = sum(hand)
        # Adjust for usable ace
        if 1 in hand and value + 10 <= 21:
            return value + 10
        return value
    
    def is_bust(self, hand):
        """Check if hand value is over 21 (bust)."""
        return self.hand_value(hand) > 21
    
    def reset(self):
        """Reset the game and deal initial cards to player and dealer."""
        self.player = [self.draw_card(), self.draw_card()]
        self.dealer = [self.draw_card(), self.draw_card()]
        return self._get_observation()
    
    def step(self, action):
        """Take an action: hit or stick."""
        if action == 'hit':
            self.player.append(self.draw_card())
            if self.is_bust(self.player):
                return self._get_observation(), -1, True  # Player busts
            else:
                return self._get_observation(), 0, False  # Continue game
        elif action == 'stick':
            while self.hand_value(self.dealer) < 17:
                self.dealer.append(self.draw_card())
            if self.is_bust(self.dealer):
                return self._get_observation(), 1, True  # Dealer busts
            elif self.hand_value(self.dealer) > self.hand_value(self.player):
                return self._get_observation(), -1, True  # Dealer wins
            elif self.hand_value(self.dealer) < self.hand_value(self.player):
                return self._get_observation(), 1, True  # Player wins
            else:
                return self._get_observation(), 0, True  # Draw
    
    def _get_observation(self):
        """Return the current game state."""
        return (self.hand_value(self.player), self.dealer[0], 1 if 1 in self.player else 0)  # (Player total, Dealer card, Usable ace)

# Monte Carlo Prediction for Blackjack
def mc_blackjack_prediction(episodes=10000, gamma=1.0):
    # Initialize environment
    env = BlackjackEnv()
    V = defaultdict(float)  # State-value function
    returns = defaultdict(list)  # Store returns for each state

    for ep in range(episodes):
        episode = []
        state = env.reset()
        done = False

        # Generate an episode
        while not done:
            action = random.choice(['hit', 'stick'])  # Random policy
            next_state, reward, done = env.step(action)
            episode.append((state, action, reward))
            state = next_state
        
        # Calculate returns and update V for first-visit
        G = 0
        visited_states = set()
        for state, action, reward in reversed(episode):
            G = gamma * G + reward
            if state not in visited_states:
                returns[state].append(G)
                V[state] = np.mean(returns[state])  # First-visit update
                visited_states.add(state)

    return V

# Run Monte Carlo prediction
V = mc_blackjack_prediction()
print(V)


defaultdict(<class 'float'>, {(18, 10, 0): -0.5970149253731343, (15, 10, 0): -0.6535433070866141, (11, 10, 0): -0.45962732919254656, (20, 5, 1): 0.047619047619047616, (19, 5, 0): -0.2765957446808511, (16, 5, 0): -0.43373493975903615, (18, 5, 0): -0.4166666666666667, (17, 5, 1): -0.14285714285714285, (21, 5, 1): 0.5882352941176471, (18, 3, 0): -0.5151515151515151, (20, 10, 0): -0.34536082474226804, (19, 10, 0): -0.4, (14, 10, 0): -0.6360424028268551, (16, 8, 0): -0.4918032786885246, (20, 6, 0): -0.020618556701030927, (15, 9, 0): -0.509090909090909, (8, 8, 0): -0.3888888888888889, (17, 5, 0): -0.31666666666666665, (8, 4, 0): -0.20833333333333334, (16, 3, 0): -0.4810126582278481, (13, 3, 0): -0.41975308641975306, (13, 9, 0): -0.581081081081081, (12, 10, 0): -0.5918367346938775, (18, 9, 0): -0.3728813559322034, (9, 10, 0): -0.6410256410256411, (21, 10, 1): 0.13917525773195877, (13, 10, 1): -0.546875, (13, 10, 0): -0.7230215827338129, (14, 4, 0): -0.3880597014925373, (16, 9, 0): -0.6125, (1

In [8]:
import numpy as np
import random
from collections import defaultdict

class GridWorldEnv:
    def __init__(self, grid_size=(4, 4), goal_state=(3, 3)):
        self.grid_size = grid_size
        self.goal_state = goal_state
        self.actions = ['up', 'down', 'left', 'right']
        self.reset()

    def reset(self):
        """Reset the environment and return the starting state."""
        self.state = (0, 0)  # Start at top-left corner
        return self.state

    def step(self, action):
        """Take a step in the environment."""
        row, col = self.state

        if action == 'up':
            next_state = (max(row - 1, 0), col)
        elif action == 'down':
            next_state = (min(row + 1, self.grid_size[0] - 1), col)
        elif action == 'left':
            next_state = (row, max(col - 1, 0))
        elif action == 'right':
            next_state = (row, min(col + 1, self.grid_size[1] - 1))
        else:
            raise ValueError(f"Unknown action: {action}")

        self.state = next_state

        if self.state == self.goal_state:
            reward = 1  # Reward for reaching the goal
            done = True
        else:
            reward = -0.1  # Small penalty for each step
            done = False

        return next_state, reward, done

    def sample_action(self):
        """Randomly sample an action."""
        return random.choice(self.actions)

def epsilon_greedy_action(state, Q, epsilon, actions):
    """Choose an action using epsilon-greedy."""
    if random.random() < epsilon:
        return random.choice(actions)  # Explore: random action
    else:
        return max(Q[state], key=Q[state].get)  # Exploit: best action based on Q-values

def generate_episode(env, Q, epsilon):
    """Generates an episode using epsilon-greedy policy."""
    episode = []
    state = env.reset()
    while True:
        action = epsilon_greedy_action(state, Q, epsilon, env.actions)
        next_state, reward, done = env.step(action)
        episode.append((state, action, reward))
        state = next_state
        if done:
            break
    return episode

def monte_carlo_control(env, episodes=1000, gamma=0.9, epsilon=0.1):
    """Performs Monte Carlo control with epsilon-greedy policy."""
    Q = defaultdict(lambda: {a: 0.0 for a in env.actions})  # Initialize Q-values
    returns = defaultdict(list)  # To store all returns for state-action pairs

    for ep in range(episodes):
        # Generate an episode
        episode = generate_episode(env, Q, epsilon)
        G = 0  # Initialize return

        # Loop backward over the episode
        for t in range(len(episode)-1, -1, -1):
            state, action, reward = episode[t]
            G = reward + gamma * G  # Calculate cumulative return

            # First visit Monte Carlo: check if state-action pair was visited earlier in episode
            if (state, action) not in [(x[0], x[1]) for x in episode[:t]]:
                returns[(state, action)].append(G)  # Store return
                Q[state][action] = np.mean(returns[(state, action)])  # Update Q-value

        # Optionally decrease epsilon over time (to reduce exploration)
        epsilon = max(epsilon * 0.99, 0.01)  # Decrease epsilon

    # Extract the optimal policy from Q-values
    policy = {state: max(Q[state], key=Q[state].get) for state in Q}
    return Q, policy

# Initialize environment and run Monte Carlo with epsilon-greedy
env = GridWorldEnv()
Q, optimal_policy = monte_carlo_control(env)

# Display the optimal policy
print("Optimal Policy:")
for state, action in optimal_policy.items():
    print(f"State {state}: Best action -> {action}")

# Display the learned Q-values
print("\nLearned Q-Values:")
for state, actions in Q.items():
    print(f"State {state}: {actions}")


Optimal Policy:
State (0, 0): Best action -> down
State (0, 1): Best action -> down
State (1, 1): Best action -> right
State (0, 2): Best action -> right
State (0, 3): Best action -> down
State (1, 3): Best action -> down
State (1, 2): Best action -> down
State (2, 3): Best action -> down
State (2, 2): Best action -> right
State (1, 0): Best action -> right
State (2, 0): Best action -> right
State (2, 1): Best action -> down
State (3, 0): Best action -> up
State (3, 1): Best action -> right
State (3, 2): Best action -> right

Learned Q-Values:
State (0, 0): {'up': -0.36918650060780034, 'down': 0.16357620687307062, 'left': -0.20283849999999978, 'right': -0.7047549991805321}
State (0, 1): {'up': -0.9999999987859735, 'down': -0.4751199976788043, 'left': -0.9999999999999994, 'right': -0.9999999999999994}
State (1, 1): {'up': -0.6063399999999995, 'down': -0.27099999999999963, 'left': -0.2460809875112374, 'right': 0.44725180993927144}
State (0, 2): {'up': -0.8764653207432103, 'down': -0.9999